In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/data/KSC/EROSION/weather-tower-export-20251707031705.csv')
display(df)


In [ ]:
df = df[df['Height']<20]
display(df)
grouped = df.groupby(['SiteName', 'Height'])
MIN_ROWS = 100
for metric in ['Barometric Pressure', 'Temperature', 'Relative Humidity', 'Average Wind Speed', 'Average Wind Direction']:
    for name, group in grouped:

        this_site = group.dropna(axis='columns', how='all')
        if metric in this_site.columns:

            if len(this_site[metric].dropna())>MIN_ROWS:
                this_site['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))
                print(name)

                if metric == 'Barometric Pressure':
                    this_site = this_site[this_site[metric] >  800]
                    this_site['PSI'] = this_site['Barometric Pressure']*0.0145038
                    this_site.plot(x='datetime', y='PSI', title=f'Site: {name}', ylabel='Barometric Pressure (PSI)', legend=False)
                else:
                    this_site.plot(x='datetime', y=metric, title=f'Site: {name}', ylabel=metric, legend=False)
                #display(this_site)

                plt.show()




In [ ]:
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

ranges = ((850,1035), (1,111), (10,99), (0,180), (0, 360))
for i,metric in enumerate(['Barometric Pressure', 'Temperature', 'Relative Humidity', 'Average Wind Speed', 'Average Wind Direction']):
    dfmetric = df.dropna(subset=[metric])
    dfmetric = dfmetric.loc[dfmetric[metric] >  ranges[i][0]]
    dfmetric = dfmetric.loc[dfmetric[metric] <  ranges[i][1]]    
    dfmetric = dfmetric.groupby('datetime')[metric].mean().reset_index()
    print(type(dfmetric))
    print(dfmetric)
    dfmetric.set_index('datetime', inplace=True)
    dfmetric = dfmetric.resample('15min').mean()
    if len(dfmetric)>MIN_ROWS:
        if metric == 'Barometric Pressure':
            dfmetric['PSI'] = dfmetric[metric]*0.0145038
            dfmetric.plot(y='PSI', ylabel=f'{metric} (PSI)', legend=False, style='-')
        else:
            dfmetric.plot(y=metric, ylabel=metric, legend=False, style='-')
        plt.show()
        dfmetric.reset_index(inplace=True)
        display(dfmetric)
        dfmetric.to_csv(f'{metric.strip()}.csv')